https://stackoverflow.com/questions/34052115/how-to-find-the-importance-of-the-features-for-a-logistic-regression-model

In [243]:
import numpy as np
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from karateclub import Graph2Vec, GL2Vec, FeatherNode, AE
import networkx as nx
import pandas as pd
from sklearn.feature_extraction import DictVectorizer
scl = StandardScaler()

In [337]:
G1 = nx.Graph()
G1.add_node(0, feature='yonke.exe')
G1.add_node(1, feature='ipnet.exe')
G1.add_node(2, feature='erop.exe' )
G1.add_node(3, feature='prob.exe' )
G1.add_edges_from([(0, 1), (1, 2)], prop='accessed')
G1.add_edges_from([(3, 2), (2, 1)], prop='lead')

G2 = nx.Graph()
G2.add_node(0, feature='yonke.exe')
G2.add_node(1, feature='ipnet.exe')
G2.add_node(2, feature='grob.exe' )
G2.add_node(3, feature='prob.exe' )
G2.add_edges_from([(0, 1), (1, 2)], prop='accessed')
G2.add_edges_from([(2, 1), (1, 3)], prop='lead')

H1 = nx.Graph()
H1.add_node(0, feature='fang.exe'  )
H1.add_node(1, feature='ipneft.exe')
H1.add_node(2, feature='grob.exe'  )
H1.add_node(3, feature='prob.exe' )
H1.add_edges_from([(0, 1), (1, 2)], prop='deleted')
H1.add_edges_from([(2, 3), (3,1)], prop='lead')

H2 = nx.Graph()
H2.add_node(0, feature='fang.exe'  )
H2.add_node(1, feature='ipneft.exe')
H2.add_node(2, feature='grob.exe'  )
H2.add_node(3, feature='prob.exe' )
H2.add_node(4, feature='prob.exe' )
H2.add_edges_from([(3, 1), (1, 3)], prop='deleted')
H2.add_edges_from([(2, 1), (1, 4)], prop='lead')


I = nx.Graph()
I.add_node(0,feature='yonke.exe'  )
I.add_node(1,feature='ipneft.exe')
I.add_node(2,feature='erop.exe' )
I.add_node(3, feature='prob.exe' )
I.add_edges_from([(0, 1), (1, 2)], prop='deleted')
I.add_edges_from([(3, 2), (2, 1)], prop='lead')

graphlist = [G1, G2, H1,H2, I]

In [341]:
gfeatlist = []

for graphasda in graphlist:
    # get node data
    allnodes = list(graphasda.nodes(data=True))
    allnodes = [x[1] for x in allnodes]

    # Vectorise feats
    vactmodel = DictVectorizer(sparse=False)
    feats = vactmodel.fit_transform(allnodes)
    print(feats)

#     # get node embed
#     nodeembmodel = FeatherNode()
#     nodeembmodel.fit(graphasda, feats)
#     graphemb = nodeembmodel.get_embedding()
#     print(graphemb)
#     # Sub node embed
#     finalemb = np.sum(graphemb, axis=1)
#     finalemb = np.squeeze(finalemb)
    
#     gfeatlist.append(finalemb)

[[0. 0. 0. 1.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]]
[[0. 0. 0. 1.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]]
[[1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [0. 1. 0. 0.]
 [0. 0. 0. 1.]]
[[1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [0. 1. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]]
[[0. 0. 0. 1.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]]


In [351]:
allnodes = [list(G.nodes(data=True)) for G in graphlist]
allnodes

[[(0, {'feature': 'yonke.exe'}),
  (1, {'feature': 'ipnet.exe'}),
  (2, {'feature': 'erop.exe'}),
  (3, {'feature': 'prob.exe'})],
 [(0, {'feature': 'yonke.exe'}),
  (1, {'feature': 'ipnet.exe'}),
  (2, {'feature': 'grob.exe'}),
  (3, {'feature': 'prob.exe'})],
 [(0, {'feature': 'fang.exe'}),
  (1, {'feature': 'ipneft.exe'}),
  (2, {'feature': 'grob.exe'}),
  (3, {'feature': 'prob.exe'})],
 [(0, {'feature': 'fang.exe'}),
  (1, {'feature': 'ipneft.exe'}),
  (2, {'feature': 'grob.exe'}),
  (3, {'feature': 'prob.exe'}),
  (4, {'feature': 'prob.exe'})],
 [(0, {'feature': 'yonke.exe'}),
  (1, {'feature': 'ipneft.exe'}),
  (2, {'feature': 'erop.exe'}),
  (3, {'feature': 'prob.exe'})]]

In [345]:
allnodes = [list(G.nodes(data=True))[1] for G in graphlist]
#allnodes = [x[1] for x in allnodes]

allnodes4 = list(H2.nodes(data=True))
allnodes4 = [x[1] for x in allnodes4]

# Vectorise feats
vactmodel = DictVectorizer(sparse=False)
vactmodel.fit(allnodes)

asdas = vactmodel.transform(allnodes4)

asdas

AttributeError: 'tuple' object has no attribute 'items'

In [335]:
train_X = gfeatlist[:4]
train_X = scl.fit_transform(train_X)
train_Y = [1,1,2,2]

test_X = [gfeatlist[4]]
test_X = scl.transform(test_X)
test_Y = [2]

In [336]:
clf = SGDClassifier(max_iter=1000, tol=1e-3)
clf.fit(train_X, train_Y)
clf.predict(test_X)

array([1])